In [1]:
import wmfdata as wmf
import time

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


Let's try out different methods for calculating new active editors on the Korean Wikipedia for April and May 2018. These are non-bot users who both registered their account and made at least 5 content edits during the calendar month.

# MediaWiki application databases

In [34]:
%%time
mdb_res = wmf.mariadb.run([
"use kowiki",    
"""
SELECT
    reg_month,
    SUM(reg_month_revisions >= 5) as new_active_editors
FROM (
    SELECT
        reg_month,
        SUM(reg_month_revisions) as reg_month_revisions
    FROM
      (
        /* Get revisions to content pages that are still visible */
        SELECT
          LEFT(user_registration, 6) as reg_month,
          user_id,
          user_name,
          SUM(
              rev_id IS NOT NULL and 
              LEFT(rev_timestamp, 6) = LEFT(user_registration, 6)
            ) AS reg_month_revisions
        FROM user
        INNER JOIN logging ON /* Filter users not created manually */
          log_user = user_id AND
          log_type = "newusers" AND
          log_action = "create"
        LEFT JOIN revision ON
            rev_user_text = user_name
        LEFT JOIN page ON
            rev_page = page_id
        WHERE 
            user_registration between "201804" and "201806" and
            rev_timestamp between "201804" and "201806" and
            page_namespace = 0
        GROUP BY 1, 2, 3

        UNION ALL

        /* Get revisions to content pages that have been archived */
        SELECT
          LEFT(user_registration, 6) as reg_month,
          user_name,
          user_id,
          SUM(
              ar_id IS NOT NULL and 
              LEFT(ar_timestamp, 6) = LEFT(user_registration, 6)
            ) AS reg_month_revisions
        FROM user
        INNER JOIN logging ON /* Filter users not created manually */
          log_user = user_id AND
          log_type = "newusers" AND
          log_action = "create"
        LEFT JOIN archive ON 
          ar_user_text = user_name
        WHERE 
            user_registration between "201804" and "201806" and
            ar_timestamp between "201804" and "201806" and
            ar_namespace = 0
        GROUP BY 1, 2, 3
      ) AS user_content_revision_count
    WHERE 
        user_id NOT IN (SELECT ug_user FROM user_groups WHERE ug_group = "bot") and
        user_id NOT IN (select ufg_user FROM user_former_groups WHERE ufg_group = "bot")
    GROUP BY user_name, reg_month
) unioned_revision_count
GROUP BY reg_month
"""])

CPU times: user 56 ms, sys: 4 ms, total: 60 ms
Wall time: 1min 29s


In [35]:
mdb_res

,reg_month,new_active_editors
0,201804,197.0
1,201805,171.0


# Editor month

In [17]:
%%time
em_res = wmf.mariadb.run(    
"""
select 
    month, 
    sum(extract(year_month from user_registration) = extract(year_month from month)) as new_active_editors
from staging.editor_month
where
    local_user_id != 0 and
    content_edits >= 5 and
    wiki = "kowiki" and
    month in ("2018-04-01", "2018-05-01") and
    bot_flag = 0 and
    convert(user_name using utf8) not regexp "bot\\\\b"
group by month;
""")

CPU times: user 60 ms, sys: 4 ms, total: 64 ms
Wall time: 294 ms


In [18]:
em_res

,month,new_active_editors
0,2018-04-01,214.0
1,2018-05-01,183.0


# Mediawiki history

In [42]:
%%time
mwh_res = wmf.hive.run("""
select
    substr(event_user_creation_timestamp, 1, 7) as month,
    sum(cast(reg_month_edits >= 5 as int)) as new_active_editors
from (
    select
        event_user_creation_timestamp,
        count(*) as reg_month_edits
    from wmf.mediawiki_history
    where
        snapshot = "2018-06" and
        event_entity = "revision" and
        event_type = "create" and
        wiki_db = "kowiki" and
        event_timestamp >= "2018-04" and
        event_timestamp < "2018-06" and
        event_user_is_created_by_system = 0 and
        page_namespace_is_content_historical and
        not (event_user_is_bot_by_name or array_contains(event_user_groups, "bot")) and
        substr(event_user_creation_timestamp, 1, 7) = substr(event_timestamp, 1, 7)
    group by event_user_text, event_user_creation_timestamp
) first_month_editors
group by substr(event_user_creation_timestamp, 1, 7)
""")

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 2min 6s


In [43]:
mwh_res

,month,new_active_editors
0,2018-04,159
1,2018-05,147
